#Secondary structure prediction

## Set up

In [ ]:
import os

try:
    import nucleotide_transformer
except:
    !pip install numpy==1.23.5
    !pip install git+https://github.com/instadeepai/nucleotide-transformer@main |tail -n 1
    import nucleotide_transformer

if "COLAB_TPU_ADDR" in os.environ:
    from jax.tools import colab_tpu

    colab_tpu.setup_tpu()

!pip install -q biopython transformers datasets huggingface_hub accelerate
!pip install lightgbm

import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model

In [ ]:
#@title Select a model
#@markdown ---
model_name = '500M_human_ref'#@param['500M_human_ref', '500M_1000G', '2B5_1000G', '2B5_multi_species', '50M_multi_species_v2', '100M_multi_species_v2', '250M_multi_species_v2', '500M_multi_species_v2']
#@markdown ---

##Import model

In [ ]:
embeddings_layer = 20  # layer of embeddings extraction (0:24)

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    embeddings_layers_to_save=(embeddings_layer,),
    max_positions=999
)
forward_fn = hk.transform(forward_fn)

##Import dataset

In [ ]:
!pip install -q biopython transformers datasets huggingface_hub accelerate
from datasets import load_dataset
import pandas as pd

dataset_dict = load_dataset("proteinea/secondary_structure_prediction")
dataset = dataset_dict["train"].to_pandas()
X_proteins = dataset["input"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
X_proteins = [x for x in X_proteins if "X" in x]
print(len(X_proteins))

66


## Preprocessing

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset_dict = load_dataset("proteinea/secondary_structure_prediction")
dataset = dataset_dict["train"].to_pandas()

def filter_dataset(dataset):
    # making subsequences of length >=40 of sequences with "X" or "O".
    filtered_rows = []

    for index, row in dataset.iterrows():
        input_string = row["input"]
        dssp3_string = row["dssp3"]
        dssp8_string = row["dssp8"]

        x_positions = [pos for pos, char in enumerate(input_string) if char == 'X']
        o_positions = [pos for pos, char in enumerate(input_string) if char == 'O']

        if not (x_positions or o_positions):
            # Keep the entire row if it doesn't contain 'X' or 'O'
            filtered_rows.append(row)
        else:
            occurrences = sorted(x_positions + o_positions)

            start_idx = 0  # First non "X" or "O"
            for i in occurrences:
                if i - start_idx >= 40:
                    new_row = {
                        "input": input_string[start_idx:i],
                        "dssp3": dssp3_string[start_idx:i],
                        "dssp8": dssp8_string[start_idx:i]
                    }
                    filtered_rows.append(new_row)
                start_idx = i + 1

            # Check for the remaining substring at the end of the input string
            if len(input_string) - start_idx >= 40:
                new_row = {
                    "input": input_string[start_idx:],
                    "dssp3": dssp3_string[start_idx:],
                    "dssp8": dssp8_string[start_idx:]
                }
                filtered_rows.append(new_row)

    # removing odd length protein sequences because of unstable embedding rappresentations
    filtered_rows = [row for row in filtered_rows if len(row["input"]) % 2 == 0]

    df = pd.concat([pd.DataFrame([row]) for row in filtered_rows], ignore_index=True)
    return df

df = filter_dataset(dataset)
df = df[:3000]  # Adjust to available memory

X_proteins = df["input"]
y_seq = df["dssp3"]  # Can be changed to "dssp8"

## Making amino acid sequences into dna sequences

In [ ]:
import random

def protein_to_rna(protein_sequence):
    amino_acid_to_codons = {
        "A": ["GCU", "GCC", "GCA", "GCG"],
        "R": ["CGU", "CGC", "CGA", "CGG", "AGA", "AGG"],
        "N": ["AAU", "AAC"],
        "D": ["GAU", "GAC"],
        "B": ["AAU", "AAC", "GAU", "GAC"],
        "C": ["UGU", "UGC"],
        "Q": ["CAA", "CAG"],
        "E": ["GAA", "GAG"],
        "Z": ["CAA", "CAG", "GAA", "GAG"],
        "G": ["GGU", "GGC", "GGA", "GGG"],
        "H": ["CAU", "CAC"],
        "I": ["AUU", "AUC", "AUA"],
        "L": ["CUU", "CUC", "CUA", "CUG", "UUA", "UUG"],
        "K": ["AAA", "AAG"],
        "M": ["AUG"],
        "F": ["UUU", "UUC"],
        "P": ["CCU", "CCC", "CCA", "CCG"],
        "S": ["UCU", "UCC", "UCA", "UCG", "AGU", "AGC"],
        "T": ["ACU", "ACC", "ACA", "ACG"],
        "W": ["UGG"],
        "Y": ["UAU", "UAC"],
        "V": ["GUU", "GUC", "GUA", "GUG"],
        "U": ["UUU"], # Note: in this version U is associated to the wrong codon
    }

    # Converts protein sequence to RNA sequence
    rna_sequence = ''
    for amino_acid in protein_sequence:
        codons = amino_acid_to_codons.get(amino_acid)
        rna_sequence += codons[random.randint(0, len(codons)-1)]  # Chose codon

    return rna_sequence

X_rna = [protein_to_rna(protein) for protein in X_proteins]

In [ ]:
def rna_to_dna(rna_sequence):
    rna_to_dna_base_pairs = {'A': 'T', 'U': 'A', 'C': 'G', 'G': 'C'}

    # Convert RNA sequence to complementary DNA sequence
    dna_sequence = ''.join(rna_to_dna_base_pairs.get(base) for base in rna_sequence)

    return dna_sequence

X_dna = [rna_to_dna(rna_sequence) for rna_sequence in X_rna]

##Tokenizing and retrieving embeddings

In [ ]:
batch_size = 8 # Adjust to available memory

def get_6mers_embeddings(sequences: list, batch_size: int):
    embeddings = []  # Stores 6mers embeddings

    # Split sequences into batches
    batched_sequences = [sequences[i:i + batch_size] for i in range(0, len(sequences), batch_size)]

    random_key = jax.random.PRNGKey(0)
    extraction_layer = "embeddings_" + str(embeddings_layer)
    cls_token_position = 0 # Position of the CLS token for every sequence

    for batch in batched_sequences:
        # Tokenize the batch
        tokens_ids = [b[1] for b in tokenizer.batch_tokenize(batch)]
        tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)

        # Infer
        outs = forward_fn.apply(parameters, random_key, tokens)

        # Get embeddings
        batch_embeddings = outs[extraction_layer]

        # Mask for padding tokens
        padding_mask = (tokens != tokenizer.pad_token_id) & (jnp.arange(tokens.shape[1]) != cls_token_position)

        # Element-wise multiplication to set embeddings corresponding to padding tokens to zero
        masked_embeddings = batch_embeddings * padding_mask[:, :, None]

        # Removing embeddings relative to CLS and padding tokens
        sum_embeddings = jnp.sum(masked_embeddings, axis=-1)
        non_zero_mask = sum_embeddings != 0.0
        non_zero_embeddings = masked_embeddings[non_zero_mask]

        embeddings.append(non_zero_embeddings)


    return jnp.concatenate(embeddings, axis=0)

In [ ]:
X = get_6mers_embeddings(X_dna, batch_size)

In [ ]:
conc_y = ''  # Concatenated sequences of target dna
conc_y = conc_y.join(y_seq)

y = [conc_y[i] + conc_y[i+1] for i in range(0, len(conc_y), 2)]  # Cartesian product of classes

## Downstream task

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

params = {
    'objective': 'multiclass',  # 'multiclass' for multi-class classification
    'metric': 'multi_error',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

clf = lgb.LGBMClassifier(**params)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

def cartesian_accuracy_score(y_pred, y_test):
    y_pred = [char for string in y_pred for char in string]
    y_test = [char for string in y_test for char in string]
    return accuracy_score(y_test, y_pred)

# Evaluate the classifier's performance
accuracy = cartesian_accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51
